In [35]:
"""
Chapter 10. Network Models. Transhipment.
Anderson, David R., Dennis J. Sweeney,
Thomas A. Williams,
Jeffrey D. Camm y Kipp Martin
Quantitative Methods for Business, 11a. Ed
"""
from itertools import product
import math

import pulp as pl


In [36]:
# adjacency_matrix
nodes = range(8)

a = [
    [0, 0, 1, 1, 0, 0, 0, 0],  # node 1 Denver
    [0, 0, 1, 1, 0, 0, 0, 0],  # node 2 Atlanta
    [0, 0, 0, 0, 1, 1, 1, 1],  # node 3 Kansas
    [0, 0, 0, 0, 1, 1, 1, 1],  # node 4 Louisville
    [0, 0, 0, 0, 0, 0, 0, 0],  # node 5 Detroit
    [0, 0, 0, 0, 0, 0, 0, 0],  # node 6 Miami
    [0, 0, 0, 0, 0, 0, 0, 0],  # node 7 Dallas
    [0, 0, 0, 0, 0, 0, 0, 0],  # node 8 New Orleans
 ]


loads = {
    'Denver': 600,   # 0
    'Atlanta': 400,  # 1
    'Kansas': 0,     # 2
    'Louisville': 0, # 3
    'Detroit': -200, # 4
    'Miami': -150,   # 5
    'Dallas': -350,  # 6
    'New Orleans': -300,  # 7
}


costs =  [
    [1_000, 1_000, 2, 3, 1_000, 1_000, 1_000, 1_000],  # node 1 Denver
    [1_000, 1_000, 3, 1, 1_000, 1_000, 1_000, 1_000],  # node 2 Atlanta
    [1_000, 1_000, 1_000, 1_000, 2, 6, 3, 6],  # node 3 Kansas
    [1_000, 1_000, 1_000, 1_000, 4, 4, 6, 5],  # node 4 Louisville
    [1_000, 1_000, 1_000, 1_000, 1_000, 1_000, 1_000, 1_000],  # node 5 Detroit
    [1_000, 1_000, 1_000, 1_000, 1_000, 1_000, 1_000, 1_000],  # node 6 Miami
    [1_000, 1_000, 1_000, 1_000, 1_000, 1_000, 1_000, 1_000],  # node 7 Dallas
    [1_000, 1_000, 1_000, 1_000, 1_000, 1_000, 1_000, 1_000],  # node 8 New Orleans
 ]


costs = pl.makeDict((loads, loads), costs, 0)
a = pl.makeDict((loads, loads), a, 0)

print(a['Denver']['Kansas'])

1


In [37]:
# problem
problem = pl.LpProblem('Transhipment', pl.LpMinimize)

# variables
send = pl.LpVariable.dicts('Send', (loads, loads), 0, None, pl.LpContinuous)
# print('\n', vars)

# objective function
problem += pl.lpSum(costs[i][j] * send[i][j] for j in loads for i in loads), 'Sum_Costs_Transportation'

# Transhipment Constraints
for k in loads:
    problem += pl.lpSum(a[k][out] * send[k][out] for out in loads) - pl.lpSum(a[enter][k] * send[enter][k] for enter in loads) == loads[k], f'num_transhipment_into_{k}'


In [38]:

solver = pl.get_solver('GUROBI') 
problem.solve(solver)

# The status of the solution is printed to the screen
print("Status:", pl.LpStatus[problem.status])


# Each of the variables is printed with it's resolved optimum value
for v in problem.variables():
    if v.varValue > 1e-10:
        print(f'{v.name}: {v.varValue:,.2f}')

print(f'Total Cost of Transportation = {pl.value(problem.objective):,.2f}')


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 8 rows, 64 columns and 24 nonzeros
Model fingerprint: 0x3d6f9bf7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 6 rows and 58 columns
Presolve time: 0.03s
Presolved: 2 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1500000e+03   1.000000e+02   0.000000e+00      0s
       1    5.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.05 seconds
Optimal objective  5.200000000e+03
Gurobi status= 2
Status: Optimal
Send_Atlanta_Louisville: 400.00
Send_Denver_Kansas: 550.00
Send_Denver_Louisville: 50.00
Send_Kansas_Dallas: 350.00
Send_Kansas_Detroit: 200.00
Send_Louisville_Miami: 150.00
Send_Louisville_New_Orleans: 300.00
Total Co